In [2]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

gemini_api_key = settings.GEMINI_API_KEY

In [3]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [18]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [19]:
def clean_text(text, _normalizes=None):
    txt = text
    try:
        normalizes = _normalizes or (
            ('\n ?\d+ ?/ ?\d+ ?\n', '\n'),
            ('[ ]{2,}', ' '),
            (' \\n', '\n'),
            #('([^\.]|\S)\n(\S)', r'\1 \2'),
            ('(\w)\n(.)', r'\1 \2'),
            ('(,)\n(.)', r'\1 \2'),
            #('()\n()', r'\1 \2'),
            ('\n\n', '\n'),
            ('-\n', '-'),
            ('^\n', ''),
            ('–', '-'),
            ('•', '*'),
            ('[“”]', '"'),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
        )

        for regex, new in normalizes:
            search = re.search(regex, text)
            while search:
                text = re.sub(regex, new, text)
                search = re.search(regex, text)
        return text
    except:
        return txt


In [ ]:
def make_prompt(text):
    prompt = ("""

Sobre o conteúdo de <CONTEXTO></CONTEXTO>

“O quê”, “como”, “onde”, “quem” e “quando aconteceu” são as questões que devem ser respondidas direta, rápida e objetivamente, no processo de transformar um acontecimento em fato noticioso.

1) Como se fosse um jornalista experiente, formado, com prática no jornalismo institucional e na comunicação legislativa, em linguagem simples para que a maioria das pessoas possa entender o texto a ser produzido, redija uma notícia totalmente objetiva e completamente restrita aos dados contidos no documento legislativo.

2) Os adjetivos devem ficar restritos às citações entre aspas dos autores da matéria legislativa. Essas citações entre aspas, se houver, devem ser retiradas da justificativa da matéria jornalística objeto da notícia.

3) O texto deve ter uma sugestão de título, que deve ter no mínimo 20 e no máximo 60 caracteres.

4) O texto não deve conter opinião ou sugestão do jornalista IA.

<CONTEXTO>{context}</CONTEXTO>

    """).format(context=text)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [38]:
def make_prompt(text):
    prompt = ("""
Como se fosse um jornalista experiente, formado, com prática no jornalismo institucional e na comunicação legislativa, com base no texto de <CONTEXTO></CONTEXTO>, em linguagem simples para que a maioria das pessoas possa entender o texto a ser produzido, redija uma notícia totalmente objetiva e completamente restrita aos dados contidos no documento legislativo.
Sua notícia deve responder às questões “O quê”, “como”, “onde”, “quem” e “quando aconteceu”, porém, não divida o texto em itens ou tópicos com base nessas questões. Alem disso, siga as seguintes regras:
1) As questões devem ser respondidas direta, rápida e objetivamente, no processo de transformar um acontecimento em fato noticioso.
2) Como se fosse um jornalista experiente, formado, com prática no jornalismo institucional e na comunicação legislativa, em linguagem simples para que a maioria das pessoas possa entender o texto a ser produzido, redija uma notícia totalmente objetiva e completamente restrita aos dados contidos no documento legislativo.
3) Os adjetivos devem ficar restritos às citações entre aspas dos autores da matéria legislativa. Essas citações entre aspas, se houver, devem ser retiradas da justificativa da matéria jornalística objeto da notícia.
4) O texto deve ter uma sugestão de título, que deve ter no mínimo 20 e no máximo 60 caracteres.
5) O texto não deve conter opinião ou sugestão do jornalista IA.

<CONTEXTO>{context}</CONTEXTO>

    """).format(context=text)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt

In [39]:
from sapl.materia.models import MateriaLegislativa

materias = MateriaLegislativa.objects.filter(id=21533)

for materia in materias[:1]:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)

prompt = make_prompt(text)

answer = model.generate_content(prompt)

In [40]:
open(f'data/004_output_{materia.numero}.txt', 'w').write(answer.text)

1114

In [41]:
resp = answer.text


In [42]:
display(Markdown(resp))

**Vereadores de Jataí solicitam Estatuto para Administrativos da Educação**

Todos os vereadores de Jataí protocolaram, em 19 de janeiro de 2025, requerimento solicitando ao Poder Executivo o envio de um projeto de lei que crie um estatuto próprio para os servidores da área administrativa da educação municipal. O documento foi encaminhado ao Prefeito Geneilton Filho de Assis e ao Procurador Geral do Município, Geraldo Caldeira Azambuja Neto. A justificativa do requerimento ressalta a importância da valorização do servidor público para a qualidade dos serviços prestados à população. Segundo o texto, a Lei 2.822/2007, que trata do Estatuto e Plano de Cargos dos Profissionais do Magistério, não abrange os funcionários que atuam nas áreas administrativa, de alimentação e higiene/limpeza. O requerimento destaca que "a alimentação escolar representa um dos alicerces do processo de aprendizagem" e que "os agentes de limpeza são importantes na educação porque ajudam a manter a higiene e a limpeza nas escolas". Os vereadores esperam que a criação do estatuto defina os direitos e deveres desses servidores.
